In [53]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
import json
import pickle
import string
from SpoilerDataset import SpoilerDataset

In [7]:
train_dataset = SpoilerDataset('data/train_reviews.json','data/word_to_id.pickle','data/id_to_word.pickle')

In [8]:
train_dataloader = DataLoader(train_dataset)

In [101]:
class SpoilerNet(nn.Module):
    def __init__(self,word_emb_dim, hidden_dim, word_vocab_size):
        super(SpoilerNet, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.word_embedding = nn.Embedding(word_vocab_size, word_emb_dim)
        self.word_gru = nn.GRU(input_size=word_emb_dim,hidden_size=hidden_dim,num_layers=2,bidirectional=True,batch_first=True)
        self.linear = nn.Linear(in_features=2*hidden_dim,out_features=hidden_dim)
        self.attention = nn.Linear(in_features=hidden_dim,out_features=1,bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        self.sentence_gru = nn.GRU(input_size=2*hidden_dim,hidden_size=hidden_dim,num_layers=2,bidirectional=True,batch_first=True)
  
        
    def forward(self,review):
        for sentence in review:
            sentence = sentence.to(self.device)
            embedded_sentence = self.word_embedding(sentence)
            gru_out, _ = self.word_gru(embedded_sentence)
            mu = self.tanh(self.linear(gru_out))
            alpha_weights = self.softmax(self.attention(mu))
            attended_vector = (alpha_weights * gru_out).sum(dim=1)
            print(attended_vector.size())
            
        

In [102]:
EPOCHS = 1
WORD_EMBEDDING_DIM = 100
HIDDEN_DIM = 50

word_vocab_size = len(train_dataset.word_to_id)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

model = SpoilerNet(WORD_EMBEDDING_DIM,HIDDEN_DIM,word_vocab_size)

if use_cuda:
    model.cuda()

In [103]:
for epoch in range(EPOCHS):
    for batch_idx, input_data in enumerate(train_dataloader):
        review = input_data[0]
        model(review)
        break

torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 100])


1046112